In [1]:
from pyscipopt import Model
import numpy as np
from itertools import combinations

In [2]:
# create np array with assets/ATSP_17.csv
data = np.loadtxt('assets/ATSP_17.csv', delimiter=';')
n = data.shape[0]

### Primer modelo

In [3]:
model_1 = Model("ATSP_1")

In [4]:
# add variables
x = np.empty((n, n), dtype=object)
for i in range(n):
    for j in range(n):
        if i != j:
            x[i, j] = model_1.addVar(vtype="B", name=f"x_{i}_{j}")
        else:
            x[i, j] = None

# add restrictions
for i in range(n):
    model_1.addCons(sum(x[i, j] for j in range(n) if j != i) == 1, name=f"out_{i}")
    model_1.addCons(sum(x[j, i] for j in range(n) if j != i) == 1, name=f"in_{i}")

for subset_length in range(2, n):
    for subset in combinations([i for i in range(n)], subset_length):
        model_1.addCons(sum(x[i, j] for i in subset for j in subset if i != j) <= subset_length)

    

In [5]:
# seteamos el objetivo

model_1.setObjective(sum(x[i, j]*data[i,j] for i in range(n) for j in range(n) if i != j), sense="minimize")

In [ ]:
# resolver

model_1.optimize()

sol = model_1.getBestSol()

model_1.writeSol(sol, 'soluciones/ejercicio_8_opcion1.sol')

### modelo 2

In [7]:
model_2 = Model("ATSP_2")

# add variables
x = np.empty((n, n), dtype=object)
for i in range(n):
    for j in range(n):
        if i != j:
            x[i, j] = model_2.addVar(vtype="B", name=f"x_{i}_{j}")
        else:
            x[i, j] = None

# add restrictions
for i in range(n):
    model_2.addCons(sum(x[i, j] for j in range(n) if j != i) == 1, name=f"out_{i}")
    model_2.addCons(sum(x[j, i] for j in range(n) if j != i) == 1, name=f"in_{i}")

In [8]:
# restricciones especificas del modelo 1

u = np.empty(shape=n, dtype=object)

for i in range(n):
    u[i] = model_2.addVar(vtype="C", name=f"u_{i}", lb=0)

for i in range(n):
    for j in range(1, n):
        if j != i:
            model_2.addCons(u[i] + x[i,j] <= u[j] + (n-1)*(1-x[i,j]))

In [9]:
model_2.setObjective(sum(x[i, j]*data[i,j] for i in range(n) for j in range(n) if i != j), sense="minimize")

In [10]:

model_2.optimize()

sol = model_2.getBestSol()

model_2.writeSol(sol, 'soluciones/ejercicio_8_opcion2.sol')